# Tensorflow 2 implementation of open-neural-apc

Copyright (c) 2020-2021, Nico Jahn <br>
All rights reserved.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# extension to run open-neural-apc in google colab and retrieving all neccessary files
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    import os

    os.system("apt -qq update && apt -qq install git-lfs")
    os.system("git lfs install")
    os.system("git clone https://github.com/nicojahn/open-neural-apc.git")
    os.chdir("open-neural-apc")

In [ ]:
%%bash
# install and update required packages
python3 -m pip install --upgrade pip -q
python3 -m pip install -r requirements.txt -q

In [ ]:
from utils import load_config, allow_growth

allow_growth()
# read the config file
# it includes more or less all hyperparameter used in the model and preprocessing/training step
data_parameter, model_parameter, training_parameter = load_config(verbose=0)

In [ ]:
# parsing command line arguments and overwriting config if those are supplied
from argument_parser import parse_arguments, overwrite_config

parsed_arguments, _ = parse_arguments()
overwrite_config(parsed_arguments, data_parameter, model_parameter, training_parameter)

In [ ]:
""" Loading the training data """
from data_loader import DataLoader

training_data = DataLoader(training_parameter, data_parameter["data"], "training")

if len(training_data) > 0:
    """Creating a model and save the config"""
    from napc import NeuralAPC

    # create model from config
    napc = NeuralAPC(model_parameter, training_parameter)
    napc.compile()
    napc.save()

    # writing config file into model folder
    import json

    new_config = {
        "data_parameter": data_parameter,
        "model_parameter": model_parameter,
        "training_parameter": training_parameter,
    }
    with open(napc.config_path, "w+") as config:
        config.write(json.dumps(new_config, sort_keys=True, indent=2))
    print(f"Model folder created and config saved: {napc.config_path}")

    """Data generator initialization"""
    from data_generator import DataGenerator

    # this is class which preprocesses the training data every epoch
    # it creates the necessary labels/bounds and augments the data
    training_generator = DataGenerator(training_data, training_parameter, training=True)

    """ Training procedure"""
    napc.fit(
        training_generator,
        epochs=training_parameter["epochs"],
        initial_epoch=napc.epoch,
        max_queue_size=len(training_generator),
        workers=4,
        use_multiprocessing=False,
        callbacks=napc.callbacks,
    )

In [ ]:
""" For all of you which don't have the training data and just want to execute the notebooks """
if len(training_data) == 0:
    data_parameter, model_parameter, training_parameter = load_config(
        "models/config.json", verbose=False
    )
    from napc import NeuralAPC

    napc = NeuralAPC(model_parameter, training_parameter)
    # Loading the included model (it has no subdirectory)
    napc.load_model(epoch=10000, model_path="models/")
    # The model_path of the model is not 'models/', but the previously created subdirectory
    # You could now train it further/save it/ etc.
    napc.compile()

In [ ]:
""" Produce videos on all validation sequences or just validate the model """
# de-/activate video creation
create_vids = False
if create_vids:
    ! apt -qq update && apt -qq install -y ffmpeg

# copy dict from training and modify the concatenation
validation_parameter = training_parameter.copy()
validation_parameter["concatenation_length"] = 1

# read the validation data
from data_loader import DataLoader

validation_data = DataLoader(validation_parameter, data_parameter["data"], "validation")

# process them (i need the bounds in y for the accuracy and the videos)
from data_generator import DataGenerator

validation_generator = DataGenerator(
    validation_data, validation_parameter, training=False
)

from tqdm.notebook import trange

accuracy = []
for batch_idx in trange(len(validation_generator), desc="Batches done", unit="batches"):
    # get batch, predict and calculate accuracy
    x, y = validation_generator[batch_idx]
    predictions = napc.predict_on_batch(x)
    accuracy += [napc.accuracy(y, predictions)]

    # creates my videos
    if create_vids:
        from video_generator import create_videos

        create_videos(
            x,
            y,
            predictions,
            napc.epoch,
            batch_idx * validation_parameter["batch_size"],
            data_parameter["class_names"],
        )

In [ ]:
# Since I'm not allowed to upload more sequences and I don't want to publish a perfect model
# (therefore, I haven't tested this one) the accuracy is just an approximation of the true capabilities
# The 'validation' data in this case is in fact a test set (last epoch was chosen without selection)
# In practice someone would use k-fold-CV and would reason about the average performance
import numpy as np

# So let's have a look how well the model does...
print(f"Accuracy: {100*np.mean(accuracy)} %")